In [ ]:
import numpy as np
import pandas as pd
import osmnx as ox
import geopandas as gpd

import pandana
print(pandana.__version__)

### 01. Prepare routable network using Pandana

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
from pandana.loaders import osm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Step 1: Retrieve street network data for Leeds
place = "Leeds, United Kingdom"
streets_graph = ox.graph_from_place(place, network_type='walk')

In [ ]:
# Plot the original graph
fig, ax = ox.plot_graph(streets_graph, node_size=0.5)

In [ ]:
edges = ox.graph_to_gdfs(streets_graph, nodes=False)
nodes = ox.graph_to_gdfs(streets_graph, edges=False)
edges_df = edges.reset_index()
nodes_df = nodes.reset_index()

In [ ]:
nodes_df.set_index('osmid', inplace=True)

In [ ]:
edges_df['from'] = edges_df['u'].copy()
edges_df['to'] = edges_df['v'].copy()

In [ ]:
edges_df = edges_df[['from', 'to', 'length']]

In [ ]:
edges_df['from'] = edges_df['from'].astype(int)
edges_df['to'] = edges_df['to'].astype(int)

In [ ]:
edges_df.dtypes

In [ ]:
edges_df.head()

In [ ]:
nodes_df.head()

In [ ]:
# Create Pandana network
network = pandana.Network(nodes['x'], nodes['y'], edges_df['from'], edges_df['to'], edge_weights = edges_df[['length']], twoway=True)

In [ ]:
#Check pandana network edges
network.edges_df.head()

In [ ]:
#Check pandana network nodes
network.nodes_df.head()

### 02. Construct land use datasets using OSM

In [ ]:
tags = {"amenity":True}
amenity_gdf = ox.geometries_from_place(place, tags)
amenity_gdf.shape

In [ ]:
amenity_sel_gdf = amenity_gdf[amenity_gdf.geom_type == 'Point']

In [ ]:
#Check unique land use types
amenities_list = amenity_sel_gdf['amenity'].unique()
print(sorted(amenities_list))

In [ ]:
#List of amenities by types
culture_list = ['arts_centre', 'dancing_school', 'dojo', 'theatre' ]
food_list = ['bar', 'cafe', 'fast_food', 'food_court', 'ice_cream', 'pub', 'restaurant', 'vending_machine']
commercial_list = ['atm','bank','bureau_de_change','car_rental', 'car_wash', 'charging_station', 'coworking_space', 'credit_union', 'money_transfer', 'post_office']
community_list = ['childcare','community_centre', 'kindergarten', 'library', 'nursing_home', 'police', 'social_centre', 'social_club', 'social_facility','townhall']
education_list = ['college', 'driving_school', 'school','university']
medical_list = ['clinic', 'dentist', 'doctors', 'hospital', 'pharmacy']
leisure_list = ['casino', 'cinema', 'events_venue', 'health_spa', 'music_venue', 'music_venue,events_venue,_bar', 'nightclub']
religion_list = ['place_of_worship']

# Filtering the amenities
culture_filtered = amenity_sel_gdf[amenity_sel_gdf['amenity'].isin(culture_list)]
food_filtered = amenity_sel_gdf[amenity_sel_gdf['amenity'].isin(food_list)]
commercial_filtered = amenity_sel_gdf[amenity_sel_gdf['amenity'].isin(commercial_list)]
community_filtered = amenity_sel_gdf[amenity_sel_gdf['amenity'].isin(community_list)]
education_filtered = amenity_sel_gdf[amenity_sel_gdf['amenity'].isin(education_list)]
medical_filtered = amenity_sel_gdf[amenity_sel_gdf['amenity'].isin(medical_list)]
leisure_filtered = amenity_sel_gdf[amenity_sel_gdf['amenity'].isin(leisure_list)]
religion_filtered = amenity_sel_gdf[amenity_sel_gdf['amenity'] == 'place_of_worship']

In [ ]:
medical_filtered.to_file("/data/leeds_medical.gpkg", layer='medical',driver="GPKG")

In [ ]:
medical_filtered.describe()

In [ ]:
culture_filtered['lat'] = culture_filtered.geometry.y
culture_filtered['lon'] = culture_filtered.geometry.x

food_filtered['lat'] = food_filtered.geometry.y
food_filtered['lon'] = food_filtered.geometry.x

commercial_filtered['lat'] = commercial_filtered.geometry.y
commercial_filtered['lon'] = commercial_filtered.geometry.x

community_filtered['lat'] = community_filtered.geometry.y
community_filtered['lon'] = community_filtered.geometry.x

education_filtered['lat'] = education_filtered.geometry.y
education_filtered['lon'] = education_filtered.geometry.x

medical_filtered['lat'] = medical_filtered.geometry.y
medical_filtered['lon'] = medical_filtered.geometry.x

leisure_filtered['lat'] = leisure_filtered.geometry.y
leisure_filtered['lon'] = leisure_filtered.geometry.x

religion_filtered['lat'] = religion_filtered.geometry.y
religion_filtered['lon'] = religion_filtered.geometry.x

In [ ]:
culture_gdf = culture_filtered.reset_index()
food_gdf = food_filtered.reset_index()
commercial_gdf = commercial_filtered.reset_index()
community_gdf = community_filtered.reset_index()
education_gdf = education_filtered.reset_index()
medical_gdf = medical_filtered.reset_index()
leisure_gdf = leisure_filtered.reset_index()
religion_gdf = religion_filtered.reset_index()

In [ ]:
culture_gdf.set_index('osmid', inplace=True)
food_gdf.set_index('osmid', inplace=True)
commercial_gdf.set_index('osmid', inplace=True)
community_gdf.set_index('osmid', inplace=True)
education_gdf.set_index('osmid', inplace=True)
medical_gdf.set_index('osmid', inplace=True)
leisure_gdf.set_index('osmid', inplace=True)
religion_gdf.set_index('osmid', inplace=True)

In [ ]:
print("DataFrame Summary:")
print(offices_gdf)

### 03. Compute accessibility to land use

**Access to Culture** 

How many of each amenities are within 800 meters of each node? Will check the results everytime. 

In [ ]:
culture_nodes = network.get_node_ids(culture_gdf.lon, culture_gdf.lat)
culture_nodes.head()

In [ ]:
network.set(culture_nodes, 
            name = 'culture')
culture_accessibility = network.aggregate(distance = 800,
                                  type = 'count',
                                  name = 'culture')
culture_accessibility.describe()

**Access to food**

In [ ]:
food_nodes = network.get_node_ids(food_gdf.lon, food_gdf.lat)
network.set(food_nodes, 
            name = 'food')
food_accessibility = network.aggregate(distance = 800,
                                  type = 'count',
                                  name = 'food')
food_accessibility.describe()

**Access to commerical**

In [ ]:
commercial_nodes = network.get_node_ids(commercial_gdf.lon, commercial_gdf.lat)
network.set(commercial_nodes, 
            name = 'commercial')
commercial_accessibility = network.aggregate(distance = 800,
                                  type = 'count',
                                  name = 'commercial')
commercial_accessibility.describe()

**Access to community**

In [ ]:
community_nodes = network.get_node_ids(community_gdf.lon, community_gdf.lat)
network.set(community_nodes, 
            name = 'community')
community_accessibility = network.aggregate(distance = 800,
                                  type = 'count',
                                  name = 'community')
community_accessibility.describe()

**Access to education**

In [ ]:
education_nodes = network.get_node_ids(education_gdf.lon, education_gdf.lat)
network.set(education_nodes, 
            name = 'education')
education_accessibility = network.aggregate(distance = 800,
                                  type = 'count',
                                  name = 'education')
education_accessibility.describe()

**Access to medical**

In [ ]:
medical_nodes = network.get_node_ids(medical_gdf.lon, medical_gdf.lat)
network.set(medical_nodes, 
            name = 'medical')
medical_accessibility = network.aggregate(distance = 800,
                                  type = 'count',
                                  name = 'medical')
medical_accessibility.describe()

**Access to leisure**

In [ ]:
leisure_nodes = network.get_node_ids(leisure_gdf.lon, leisure_gdf.lat)
network.set(leisure_nodes, 
            name = 'leisure')
leisure_accessibility = network.aggregate(distance = 800,
                                  type = 'count',
                                  name = 'leisure')
leisure_accessibility.describe()

**Access to religion**

In [ ]:
religion_nodes = network.get_node_ids(religion_gdf.lon, religion_gdf.lat)
network.set(religion_nodes, 
            name = 'religion')
religion_accessibility = network.aggregate(distance = 800,
                                  type = 'count',
                                  name = 'religion')
religion_accessibility.describe()

In [ ]:
# Check the ouput using the examples from pandana demo
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
fig, ax = plt.subplots(figsize=(11,7))

plt.title('Leeds: Food POI within 800m')
plt.scatter(network.nodes_df.x, network.nodes_df.y, 
            c=commercial_accessibility, s=1, cmap='YlOrRd', 
            norm=mcolors.BoundaryNorm(boundaries=np.linspace(min(commercial_accessibility), max(commercial_accessibility), 11), 
                                      ncolors=256),
                      edgecolor='none')  # Remove black outlines
cb = plt.colorbar()
cb.outline.set_edgecolor('none')
plt.show()

In [97]:
# Add data to network.nodes_df
network.nodes_df['culture_choice_800m'] = culture_accessibility
network.nodes_df['commercial_choice_800m'] = commercial_accessibility
network.nodes_df['food_choice_800m'] = food_accessibility
network.nodes_df['community_choice_800m'] = community_accessibility
network.nodes_df['education_choice_800m'] = education_accessibility
network.nodes_df['leisure_choice_800m'] = leisure_accessibility
network.nodes_df['religion_choice_800m'] = religion_accessibility
network.nodes_df['medical_choice_800m'] = medical_accessibility

# Save it as csv
network.nodes_df.to_csv('/data/leeds_nodes.csv')

### 04. Compute nearest distances to amenities 
(Not used for the final richness index but useuful to have when neeed)

In [ ]:
# Precompute catchment
network.precompute(distance=800)  # Calculate accessibility within a distance of 800 meters

# Set pois
network.set_pois("offices", 800, 10, offices_gdf['lon'], offices_gdf['lat'])

# Get the distance to the nearest offices nodes
nearest_offices_distances = network.nearest_pois(distance=800, category='offices')

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

plt.title('Leeds: Distance to the nearest office')
plt.scatter(network.nodes_df.x, network.nodes_df.y, 
            c=nearest_offices_distances, s=1, cmap='YlOrRd', 
            norm=matplotlib.colors.LogNorm())
cb = plt.colorbar()

plt.show()

### 05. Get building layers

In [ ]:
# Define the tag to query for buildings
tags = {"building": True}
place = "Leeds, United Kingdom"
# Query for building footprints within Amsterdam
building_gdf = ox.geometries_from_place(place, tags=tags)

In [ ]:
print(type(building_gdf))

In [ ]:
# Filter out rows with 'node' as element_type
buildings_filtered = building_gdf.loc[building_gdf.index.get_level_values('element_type') != 'node']

In [ ]:
buildings_filtered.plot()

In [ ]:
buildings_filtered = buildings_filtered.reset_index()
buildings_gdf = buildings_filtered.set_index('osmid')

In [ ]:
buildings_gdf_projected = ox.projection.project_gdf(buildings_filtered)

In [ ]:
buildings_gdf = buildings_gdf_projected.set_crs('EPSG:27700', allow_override=True)
buildings_gdf.crs

### 06. Compute land use richness & Join nodes attributes to buildings for visualisation 
(QGIS might be more stable to run nearest join for 300k buildings)

In [99]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
from shapely.ops import nearest_points
from shapely.strtree import STRtree
from shapely.geometry import Point
# Define the CRS 
crs = 'EPSG:27700'

# Read the CSV file with longitude and latitude data using geopandas
amenities_gdf = pd.read_csv('/data/leeds_nodes.csv')

# Create a geometry column with Point objects from longitude (x) and latitude (y)
geometry = [Point(xy) for xy in zip(amenities_gdf['x'], amenities_gdf['y'])]

# Create a GeoDataFrame with the geometry column and CRS
amenities_gdf = gpd.GeoDataFrame(amenities_gdf, geometry=geometry, crs=crs)

In [ ]:
amenities_gdf.crs

In [ ]:
# Create a spatial index for nodes_gdf
nodes_tree = STRtree(amenities_gdf.geometry)

In [ ]:
#Using buildings layer cleaned in the earlier step
buildings_joined_gdf = buildings_gdf.sjoin_nearest(amenities_gdf)

In [ ]:
buildings_joined_gdf.head()

In [ ]:
# Define a function to calculate the richness score
def calculate_richness(row):
    columns = ['culture_choice_800m','commercial_choice_800m', 'food_choice_800m', 'community_choice_800m', 'education_choice_800m', 'leisure_choice_800m', 'religion_choice_800m', 'medical_choice_800m']
    return sum(1 for col in columns if pd.notnull(row[col]) and row[col] != 0)

buildings_joined_gdf['richness'] = buildings_joined_gdf.apply(calculate_richness, axis=1)

print(buildings_joined_gdf)

In [14]:
# Find columns with 'list' data type to clean the joined layer so that it can be exported to shp, geopackage etc.
columns_with_lists = [col for col, dtype in buildings_gdf.dtypes.items() if dtype == 'object' and isinstance(buildings_gdf[col].iloc[0], list)]

# Drop columns with 'list' data type
buildings_gdf = buildings_gdf.drop(columns_with_lists, axis=1)

In [ ]:
columns_with_lists

In [28]:
#otherwise export buildings and join in Q
columns_to_keep = ['geometry', 'building']
buildings_final_gdf = buildings_gdf[columns_to_keep]

In [ ]:
buildings_final_gdf.head()

In [31]:
# Save the GeoDataFrame to a GeoPackage file
buildings_final_gdf.to_file("/data/leeds_buildings.gpkg", layer='buildings_leeds',driver="GPKG")